### Import

In [1]:
#conda install -c conda-forge pyspark #conda pyspark 설치
#기타 요청되는 패키지 참고 : https://spark.apache.org/docs/latest/api/python/getting_started/install.html

In [2]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

### Init pyspark

In [3]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint') -- Dir 경로 설정
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

### Load data

In [4]:
#https://grouplens.org/datasets/movielens/ 데이터 출처
#추후에 우리 데이터에 적용
movies = spark.read.csv("movies.csv",header=True)
ratings = spark.read.csv("ratings.csv",header=True)

In [5]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [6]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [7]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### Calculate sparsity

In [8]:
# rating 데이터 셋의 개수
numerator = ratings.select("rating").count()

# 유저의 수와 영화의 수
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

print("Rate Data Count : %d" % numerator)
print("User Data Count : %d" % num_users)
print("Movie Data Count : %d" % num_movies)

# 전체 발생 가능 데이터의 개수
denominator = num_users * num_movies

# 전체 발생 데이터 대비 현재 데이터 비율 계산
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

Rate Data Count : 100836
User Data Count : 610
Movie Data Count : 9724
The ratings dataframe is  98.30% empty.


### Interpret ratings

In [9]:
# 유저의 작성 리뷰 수
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [10]:
# 영화의 작성 리뷰 수
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



## ALS 모델링 및 측정

In [11]:
# Import
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [12]:
# Test, Train 셋 나누기
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 2017152017)

# ALS 모델링
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")
als

ALS_4ee59f779e0d

In [13]:
# grid의 Hyperparam 조정(수시로 조정하며 비교해보자)
#다수의 모델로 예측값을 근사화 하기 위함
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 50, 100, 150, 200]) \
    .addGrid(als.regParam, [.01, .05, .1, .15]) \
    .build()
           
# RMSE 유사도 측정하고 이를 기반으로 예측
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print("테스트 모델 : %d개" %len(param_grid))

테스트 모델 : 20개


In [14]:
# Cross validation
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
print(cv)

CrossValidator_4a635c70b019


### 최적 모델 및 파라미터 찾기

In [16]:
#*************!!!주의!!! ML 라이브러리 모델링 연산 작업 오래 걸림 !!!!********

# Cross Validation한 als모델로 학습(fitting)
model = cv.fit(train)

#최적 모델 확인(이 부분이 PySpark ML 가장 많이 의존하는 부분)
bestmodel = model.bestModel
print(bestmodel)

#최적 모델 파라미터(Best Param)
print("Rank : ", bestmodel._java_obj.parent().getRank())
print("MaxIter : ", bestmodel._java_obj.parent().getMaxIter())
print("RegParam : ", bestmodel._java_obj.parent().getRegParam())

ALSModel: uid=ALS_4ee59f779e0d, rank=50
Rank :  50
MaxIter :  10
RegParam :  0.15


In [18]:
# 예측 평가 (RMSE 지표)
test_predictions = bestmodel.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8675016386208575


In [19]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|   1197|   3.0| 3.6430426|
|   148|   4308|   4.0|  3.014829|
|   148|   4896|   4.0| 3.4921103|
|   148|  54001|   4.0| 3.7169118|
|   148|  69757|   3.5|  3.639356|
|   148|  79091|   3.5| 3.4785583|
|   148|  88125|   4.0| 3.6607394|
|   148|  89745|   4.0|  3.397693|
|   148|  98491|   5.0| 3.7659245|
|   148| 122882|   4.0|  3.388848|
|   148| 152081|   4.0| 3.4634688|
|   463|    552|   3.5| 3.2064023|
|   463|   1320|   4.0| 3.2814095|
|   463|   2019|   4.0|  4.094052|
|   463|   2167|   3.0| 3.5864704|
|   463|   4310|   3.0|  3.186986|
|   463|   6377|   3.5|  3.840336|
|   471|   8961|   3.5| 3.5173879|
|   496|   1221|   4.0| 3.8432426|
|   496|   5952|   4.0| 3.4748962|
+------+-------+------+----------+
only showing top 20 rows

